In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pulp
import numpy as np
import pandas as pd
import time
import warnings

from itertools import product
from statistics import stdev, mean

from utils import *

In [3]:
class PatternEnumerationSolution:
    def __init__(self, problem: Problem, linear_relief: bool = False):
        self.problem = problem
        self.linear_relief = linear_relief
        self.formula, self.x, self.A = self._formulate()
        self.result = self._solve()


    def _formulate(self):
        n = self.problem.n
        B = self.problem.B
        s = self.problem.s

        patterns = []
        for pattern in product((True, False), repeat=n):
            if sum(pattern) == 0:  # 箱になにも詰めない場合
                continue
            if s[np.array(pattern)].sum() <= B:
                patterns.append(pattern)
        n_patterns = len(patterns)
        
        if self.linear_relief:
            x = np.array([pulp.LpVariable(f"x({j})", lowBound=0) for j in range(n_patterns)])
        else:
            x = np.array([pulp.LpVariable(f"x({j})", cat="Binary") for j in range(n_patterns)])
        A = np.array(patterns, dtype=int).T
        
        formula = pulp.LpProblem()
        formula += pulp.lpSum(x)
        for i in range(n):
            formula += pulp.lpDot(A[i], x) == 1, f"アイテム{i}はいづれかのパターンに1つだけ含まれる"
        
        return formula, x, A
    

    def _solve(self):
        start_time = time.time()
        self.formula.solve(pulp.PULP_CBC_CMD(msg = False))
        solve_time = time.time() - start_time

        if self.linear_relief:
            self.formula.roundSolution()

        if pulp.LpStatus[self.formula.status] != "Optimal":
            warnings.warn("最適解は得られていません。")
        
        convert_value = np.vectorize(lambda x: x.value())
        x_int = convert_value(self.x).astype(int)
        selected_idxs = np.where(x_int >= 1)[0]
        
        records = []
        for i, j in enumerate(selected_idxs):
            record = [str(i)] + (self.A[:, j] * self.problem.s).tolist()
            records.append(record)
        allocation = pd.DataFrame.from_records(records, columns=["bin"] + [f"item({i})" for i in range(self.problem.n)])

        return {
            "status": pulp.LpStatus[self.formula.status],
            "objective": self.formula.objective.value(),
            "allocation": allocation,
            "solve_time_sec": solve_time
        }

## パターン列挙（線形緩和なし）

In [4]:
solution = PatternEnumerationSolution(ProblemTest)
draw_allocation(solution.result)

In [5]:
problems = create_problems(10, 8)
solutions = [PatternEnumerationSolution(problem) for problem in problems]

time_arr = list(map(lambda x: x.result["solve_time_sec"], solutions))
print(f"mean: {mean(time_arr):.3} s\nstd: {stdev(time_arr):.3} s")

mean: 0.0167 s
std: 0.00771 s


In [6]:
print(problems[0].s)
draw_allocation(solutions[0].result)

[5 8 6 1 4 4 4 8]


## パターン列挙（線形緩和あり）
- 0 < x < 1の場合は1に丸め込み

In [7]:
solution = PatternEnumerationSolution(ProblemTest, linear_relief=True)
draw_allocation(solution.result)

In [8]:
problems = create_problems(10, 8)
solutions = [PatternEnumerationSolution(problem, linear_relief=True) for problem in problems]

time_arr = list(map(lambda x: x.result["solve_time_sec"], solutions))
print(f"mean: {mean(time_arr):.3} s\nstd: {stdev(time_arr):.3} s")

mean: 0.0114 s
std: 0.00294 s


### うまく後処理できていない例
決定変数の四捨五入処理を行っている都合で、アイテムの出てこないアイテムが存在する

In [9]:
print(problems[0].s)
draw_allocation(solutions[0].result)

[5 8 6 1 4 4 4 8]
